In [ ]:
# Task 3 - Model Explainability using SHAP - Goal: Interpret the best-performing model using SHAP

In [1]:
# 1. Imports
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# 2. Load the dataset
df = pd.read_csv('../data/processed/engineered_fraud_data.csv')
df.head()

,age,class,ip_integer,hour_of_day,day_of_week,time_since_signup,signup_to_purchase_sec,is_weekend,is_night,ip_range,avg_tx_time_user,tx_density,browser_freq,source_Direct,source_SEO,sex_M,log_purchase_value,log_tx_count
0,31,0,2516971819,3,2,1515321,1515300.0,0,1,131071,1515321.0,6.599262e-07,52560,False,True,True,3.367296,0.693147
1,36,0,2796231688,23,6,10255723,10255740.0,0,1,8388607,10255723.0,9.750653e-08,21034,False,True,False,4.094345,0.693147
2,39,0,3589372451,0,4,998469,998460.0,0,1,32767,998469.0,1.001533e-06,31364,True,False,True,3.713572,0.693147
3,21,0,1197772333,20,5,5625916,5625900.0,0,0,2097151,5625916.0,1.777488e-07,52560,False,True,True,3.688879,0.693147
4,42,0,1299832807,6,6,2165326,2165340.0,0,0,262143,2165326.0,4.618242e-07,21074,False,True,False,2.564949,0.693147


In [3]:
# 3. Feature and label split
X = df.drop(columns=['class'])
y = df['class']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# 2. Load the trained model
model = joblib.load('../models/random_forest_model.pkl')

In [ ]:
# 3. Create SHAP Explainer

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
# 4. Global Interpretability

# Summary plot (shows global feature importance)
shap.summary_plot(shap_values[1], X_test)

# Optional: Bar plot (ranking by mean absolute SHAP values)
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
# 5. Local Interpretability (For a specific sample)

# Pick a sample (e.g., first instance of the test set)
sample_index = 0
sample = X_test.iloc[[sample_index]]

# Force plot (shows impact of features on the prediction)
shap.initjs()
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][sample_index],
    sample,
    matplotlib=True  # If running locally
)

##### **6. Interpretation Summary (To include in your report)**
📝 Interpretation Summary:

1. The SHAP **summary plot** shows which features contribute the most to predicting fraud:
   - Features like 'Amount', 'Hour', and PCA components (e.g., 'V14', 'V17') appear to have high influence.
   - High 'Amount' transactions and certain PCA component patterns are strongly associated with fraud cases.

2. The **force plot** visualizes how each feature influences the model’s decision for a specific sample:
   - Features pushing the prediction toward fraud are in red.
   - Features pushing it toward non-fraud are in blue.

These plots help us understand that the model captures important patterns in time and transaction behavior that differentiate fraudulent transactions from normal ones.